In [18]:
include("markov-drum-utilities.jl")  #loads AugMatrix, AugDblMatrix, insert, functionName, abptest, makeppheader

makeppheader (generic function with 1 method)

In [19]:
#input a vector corresponding to a probability distribution and output an index according to that distribution
function markovrand(nextv)
    if round(sum(nextv), digits = 6) != 1.0
        error(nextv, " is not a stochastic vector")
    end
    maxstates = length(nextv) - 1
    r = rand(Float64)
    current = maxstates
    for i = 0:maxstates
        if r <= nextv[i + 1]
            current = i
            break
        else
            r = r - nextv[i + 1]
        end
    end
    return(current)
end


markovrand (generic function with 1 method)

In [20]:
#output bass note codes in $beats pairs of sixteenth notes
function ghostbassgen_pairs(beats::Int, trans::AugMatrix)
    transmat = trans.mat
    iv = trans.init
    nextv = iv
    L = []
    for j = 1:beats
        current = markovrand(nextv)
#print(current, " ")
        append!(L, [current])
        nextv = transmat[:, current + 1]
    end
    return(L)
end

ghostbassgen_pairs (generic function with 1 method)

In [21]:
#output bass note codes in 2*$beats notes
function ghostbassgen(beats::Int, trans::AugMatrix)
    transmat = trans.mat
    iv = trans.init
    nextv = iv
    L = []
    for j = 1:beats
        current = markovrand(nextv)
#print(current, " ")
        bincurrent = digits(current, base = 3, pad = 2)
#print(bincurrent, "\n")
        append!(L, [bincurrent[2], bincurrent[1]])
        nextv = transmat[:, current + 1]
    end
    return(L)
end

ghostbassgen (generic function with 1 method)

In [22]:
#output backbeat in $beats pairs of sixteenth notes 
function backbeatgen(beats::Int)
    L = []
    for j = 1:beats
        if j % 8 == 3
            append!(L, [2])
        elseif j % 8 == 7
            append!(L, [2])
        else
            append!(L, [0])
        end
    end
    return(L)
end

backbeatgen (generic function with 1 method)

In [23]:
#test
backbeatgen(8)

8-element Vector{Any}:
 0
 0
 2
 0
 0
 0
 2
 0

In [24]:
function lookup8ride(; variation="none")::Matrix
    r = Matrix(undef, 4, 4)
    r[1,1] = ["!N![_e","[","!N![_e","[","!N![_e","[","!N![_e","["] #hihat Quarters, trash #2
    #NB Windows midi doesn't have a good way of making a trash hat?  !N! defined as tenuto mark and lowered volume in makeppheader
    r[2,1] = ["[^e","[^e","[^e","[^e","[^e","[^e","[^e","[^e"] #hihat 8, #1
    r[3,1] = ["!>![^e","!U![^e","!>![^e","!U![^e","!>![^e","!U![^e","!>![^e","!U![^e"] #drive #8, !U! defined as antiaccent mark
    if variation == "funk"
#        r[4,1] = ["[^e","[^e","[^e","[^e","[^e","[_e","[^e","[_e"]  #slide last 2 ands #7
        r[4,1] = ["[^e","[_e","[^e","[^e","[^e","[_e","[^e","[^e"]  #"James Brown" slides #7
    elseif variation == "disco"
         r[4,1] = ["[^e","[_e","[^e","[_e","[^e","[_e","[^e","[_e"] #slide every and #6
    else
        r[4,1] = ["[^e","[^e","[^e","[^e","[^e","[^e","[^e","[_e"] #slide last and #7
    end
    r[1,2] = ["[^g","[","[^g","[","[^g","[","[^g","["] #ride quarter #2
    r[2,2] = ["[^g","[^g","[^g","[^g","[^g","[^g","[^g","[^g"] #ride 8, #1
    r[3,2] = ["[^g^D","[^g","[^g^D","[^g","[^g^D","[^g","[^g^D","[^g"] #ride 8 with step beats, #3
    r[4,2] = ["[^g","[^g^D","[^g","[^g^D","[^g","[^g^D","[^g","[^g^D"] #ride 8 with step ands, #4
    r[1,3] = ["[g","[","[g","[","[g","[","[g","["] #bell Q, #2
    r[2,3] = ["[g","[g","[g","[g","[g","[g","[g","[g"] #bell 8, #1
    r[3,3] = ["[g","[^g","[g","[^g","[g","[^g","[g","[^g"] #bell drive, offbeat ride
    r[4,3] = ["[","[g","[","[g","[","[g","[","[g"] #bell ands, #5
    r[1,4] = ["[^D","[","[^D","[","[^D","[","[^D","["]  #step Q, #2
    r[2,4] = ["[^D","[^D","[^D","[^D","[^D","[^D","[^D","[^D"] #step 8, #1
    r[3,4] = ["[g^D","[^D","[g^D","[^D","[g^D","[^D","[g^D","[^D"] #step 8 with bell beats, #9
    r[4,4] = ["[^D","[g^D","[^D","[g^D","[^D","[g^D","[^D","[g^D"] #step 8 with bell ands
    return(r)
end


lookup8ride (generic function with 1 method)

In [25]:
function lookup16ride(; variation="none")::Matrix
    r = Matrix(undef, 4, 4)
    r[1,1] = ["!N![_e","[","!N![_e","[","!N![_e","[","!N![_e","[","!N![_e","[","!N![_e","[","!N![_e","[","!N![_e","["] #hihat eighths, trash
    #NB Windows midi doesn't have a good way of making a trash hat?  !N! defined as tenuto mark and lowered volume in makeppheader
    r[2,1] = ["[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e"]; #hihat 16
    r[3,1] = ["!>![^e", "!U![^e", "!>![^e", "!U![^e", "!>![^e", "!U![^e","!>![^e", "!U![^e","!>![^e", "!U![^e", "!>![^e", "!U![^e", "!>![^e", "!U![^e", "!>![^e", "!U![^e"]; #drive 16
    if variation == "funk"
        r[4,1] = ["[^e", "[^e", "[^e", "[_e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[_e", "[^e", "[^e", "[^e", "[^e"]  #"James Brown" slides #7
    elseif variation == "disco"
         r[4,1] = ["[^e", "[^e", "[_e", "[^e", "[^e", "[^e", "[_e", "[^e", "[^e", "[^e", "[_e", "[^e", "[^e", "[^e", "[_e", "[^e"] #slide every and #6
    else
        r[4,1] = ["[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[_e"];  #one slide 16
    end
    r[1,2] = ["[^g","[","[^g","[","[^g","[","[^g","[","[^g","[","[^g","[","[^g","[","[^g","["] #ride quarter
    r[2,2] = ["[^g", "[^g", "[^g", "[^g", "[^g", "[^g", "[^g", "[^g", "[^g", "[^g", "[^g", "[^g", "[^g", "[^g", "[^g", "[^g"]; #ride 16 
    r[3,2] = ["[^g^D", "[^g", "[^g", "[^g", "[^g^D", "[^g", "[^g", "[^g", "[^g^D", "[^g", "[^g", "[^g", "[^g^D", "[^g", "[^g", "[^g"]; #ride 16 with step beats
    r[4,2] = ["[^g", "[^g", "[^g^D", "[^g", "[^g", "[^g", "[^g^D", "[^g", "[^g", "[^g", "[^g^D", "[^g", "[^g", "[^g", "[^g^D", "[^g" ]; #ride 16 with step ands
    r[1,3] = ["[g","[","[g","[","[g","[","[g","[","[g","[","[g","[","[g","[","[g","["] #bell 8
    r[2,3] = ["[g","[g","[","[","[g","[g","[","[","[g","[g","[","[","[g","[g","[","["] #bell 16th pairs on beat
    r[3,3] = ["[g","!U![^g","[g","!U![^g","[g","!U![^g","[g","!U![^g","[g","!U![^g","[g","!U![^g","[g","!U![^g","[g","!U![^g"] #bell drive, offbeat ride
    r[4,3] = ["[","[","[g","[g","[","[","[g","[g","[","[","[g","[g","[","[","[g","[g"] #bell pairs on and
    r[1,4] = ["[^D","[","[","[","[^D","[","[","[","[^D","[","[","[","[^D","[","[","["]  #step Q
    r[2,4] = ["[^D","[","[^D","[","[^D","[","[^D","[","[^D","[","[^D","[","[^D","[","[^D","["] #step 8
    r[3,4] = ["[g^D","[g","[^D","[","[g^D","[g","[^D","[","[g^D","[g","[^D","[","[g^D","[g","[^D","["] #step 8 with bell beat pair
    r[4,4] = ["[^D","[","[g^D","[g","[^D","[","[g^D","[g","[^D","[","[g^D","[g","[^D","[","[g^D","[g"] #step 8 with bell and pair
    return(r)
end


lookup16ride (generic function with 1 method)

In [52]:
function lookup8backbass()::Vector
    c = Vector(undef, 4)
     #nothing, rest-kick, rest-snare, kick-rest, kick-kick, kick-snare, snare-rest, snare-kick, snare-snare
    c[0+1] = ["]y", "]/2[F/2]", "]/2[!Q!c/2]", "F]y", "F]/2[F/2]", "F]/2[!Q!c/2]", "]", "]", "]"] #no backbeat
    c[1+1] = ["]", "]/2[cF]/2", "]/2[c]/2",  "]", "F]/2[cF]/2", "F]/2[c/2]", "]", "]", "]"]  #backbeat off 8th
    c[2+1] = ["]", "]", "]", "cF]y", "cF]/2[F/2]", "cF]/2[!Q!c/2]", "c]y", "c]/2[F/2]", "c]/2[!Q!c/2]"] #backbeat on 8th
    c[3+1] = ["]", "]", "]", "cF]/[c]/2", "cF]/2[cF]/2", "cF]/2[c/2]", "c]/2[c]/2", "c]/2[cF/2]", "c]/2[c/2]"] #backbeat on both, probably a bad idea
    return(c)
end

lookup8backbass (generic function with 1 method)

In [57]:
#input a sixteenth note bass rhythm and an 8th note ride pattern, output abc notation
function abctime8patinst16displace(L, pattern, instr; variation="none")
    n = length(L)
    r = lookup8ride(variation=variation)
    c = lookup8backbass()
    b = backbeatgen(n)  ####THIS SHOULD BE SOMEWHERE ELSE!!!!!
    m = [c[b[i] + 1][L[i] + 1] for i = 1:n]
    s = ""
    for i = 1:n
        s = s * r[pattern, instr][(i - 1) % 8 + 1] * m[i]
        if i % 4 == 0
            s = s * " "
        end
        if i % 8 == 0
            s = s * "|\n"
        end
    end
    return(s)
end


abctime8patinst16displace (generic function with 1 method)

In [28]:
#input a sixteenth note bass rhythm and an 8th note shuffled ride pattern, output abc notation
function abctime12patinst24ghost(L, pattern, instr; variation="none")
    n = length(L)
    r = lookup8ride(variation=variation)
    c = ["]y", "]/2[F/2]", "]/2[!Q!c/2]", "F]y", "F]/2[F/2]", "F]/2[!Q!c/2]"] #nothing, rest-kick, rest-ghost snare, kick-rest, kick-kick, kick-ghost
 #should never try to index more than 6 elements of c!
    m = [c[L[i] + 1] for i = 1:n]
    s = ""
    for i = 1:n
        if i % 2 == 1
            s = s * "(3:2"
        end
        s = s * r[pattern, instr][(i - 1) % 8 + 1] * m[i]
        if i % 2 == 1 
            s = s * "z" #rest
        elseif i % 2 == 0
            s = s * " "
        end
        if i % 8 == 0
            s = s * "|\n"
        end
    end
    return(s)
end


abctime12patinst24ghost (generic function with 1 method)

In [29]:
#input a sixteenth note bass rhythm and an 16th note ride pattern, output abc notation
function abctime16patinst16ghost(L, pattern, instr; variation="none")
    n = length(L)
    r = lookup16ride(variation=variation)
    c = ["]/2", "F]/2", "!Q!c]/2"] #nothing, kick, ghost snare
    m = [c[L[i] + 1] for i = 1:n]
    s = ""
    for i = 1:n
        s = s * r[pattern, instr][(i - 1) % 16 + 1] * m[i]
        if i % 4 == 0
            s = s * " "
        end
        if i % 16 == 0
            s = s * "|\n"
        end
    end
    return(s)
end


abctime16patinst16ghost (generic function with 1 method)

In [30]:
#input a shuffled 16th bass rhythm and a shuffled 16th ride pattern, output abc notation
function abctime24patinst24ghost(L, pattern, instr; variation="none")
    n = length(L)
    r = lookup16ride(variation=variation)
    c = ["]/2", "F]/2", "!Q!c]/2"] #nothing, kick, ghost snare
    m = [c[L[i] + 1] for i = 1:n]
    s = ""
    for i = 1:n
        if i % 4 == 1
            s = s * "(6:4"
        end
        s = s * r[pattern, instr][(i - 1) % 16 + 1] * m[i]
        if i % 2 == 1 
            s = s * "z/2" #rest
        elseif i % 4 == 0
            s = s * " "
        end
        if i % 16 == 0
            s = s * "|\n"
        end
    end
    return(s)
end


abctime24patinst24ghost (generic function with 1 method)

In [31]:
#output linear fill in $beats number of smallest subdivision
function fillgen(
  beats::Int,
  trans::AugDblMatrix)
    nextv = trans.init
    L = []
    for j = 1:beats
        current = markovrand(nextv)
        append!(L,current)
        if j % 2 == 1
            nextv = trans.mat1[1:6, current + 1]
        else
            nextv = trans.mat2[1:6, current + 1]
        end
    end
    return(L)
end


fillgen (generic function with 1 method)

In [32]:
#input a sixteenth note fill and a crescendo/descr indicator, output abc notation
function abcfill16(L, crdc)
    n = length(L)
    c = ["c/2","e/2","A/2","G/2","F/2","z/2"]
    m = [c[L[i] + 1] for i = 1:n]
    s = ""
    for i = 1:n
        s = s * m[i]
        if i % 4 == 0
            s = s * " "
        end
        if i % 16 == 12
            if crdc < 0
                s = s * "!>(!"
            elseif 0 < crdc
                s = s * "!<(!"
            end
        end
        if i % 16 == 14
            if crdc < 0
                s = s * "!>)!"
            elseif 0 < crdc
                s = s * "!<)!"
            end
        end
    end
    return(s)
end


abcfill16 (generic function with 1 method)

In [33]:
#input a eighth triplet fill and a crescendo/descr indicator, output abc notation
function abcfill12(L, crdc)
    n = length(L)
    c = ["c","e","A","G","F","z"]
    m = [c[L[i] + 1] for i = 1:n]
    s = ""
    for i = 1:n
        if i % 3 == 1
            s = s * "(3:2"
        end
        s = s * m[i]
        if i % 3 == 0
            s = s * " "
        end
        if i % 12 == 8
            if crdc < 0
                s = s * "!>(!"
            elseif 0 < crdc
                s = s * "!<(!"
            end
        end
        if i % 12 == 10
            if crdc < 0
                s = s * "!>)!"
            elseif 0 < crdc
                s = s * "!<)!"
            end
        end
    end
    return(s)
end

abcfill12 (generic function with 1 method)

In [34]:
#input a sixteenth triplet fill and a crescendo/descr indicator, output abc notation
function abcfill24(L, crdc)
    n = length(L)
    c = ["c/2","e/2","A/2","G/2","F/2","z/2"]
    m = [c[L[i] + 1] for i = 1:n]
    s = ""
    for i = 1:n
        if i % 6 == 1
            s = s * "(6:4"
        end
        s = s * m[i]
        if i % 6 == 0
            s = s * " "
        end
        if i % 24 == 16
            if crdc < 0
                s = s * "!>(!"
            elseif 0 < crdc
                s = s * "!<(!"
            end
        end
        if i % 24 == 20
            if crdc < 0
                s = s * "!>)!"
            elseif 0 < crdc
                s = s * "!<)!"
            end
        end
    end
    return(s)
end

abcfill24 (generic function with 1 method)

In [37]:
#output time, fill, time, fill with cresc/decr in abc notation
function abctune_multitime_displace(
  timesig,
    vol,
  bass,
  pat,
  inst,
  fill,
  crdc;
    variation="none")
    fp = ["!pp!","!p!","!mp!","!mf!","!f!","!ff!"]
    if timesig === 8 
        abctime = abctime8patinst16displace(bass, pat, inst, variation=variation)
        abcfill0 = abcfill16(fill, 0)
        abcfill1 = abcfill16(fill, crdc)
    elseif timesig == 16
        abctime = abctime16patinst16displace(bass, pat, inst, variation=variation)
        abcfill0 = abcfill16(fill, 0)
        abcfill1 = abcfill16(fill, crdc)
    elseif timesig == 12
        abctime = abctime12patinst24displace(bass, pat, inst)
        abcfill0 = abcfill12(fill, 0)
        abcfill1 = abcfill12(fill, crdc)
    elseif timesig == 24
        abctime = abctime24patinst24displace(bass, pat, inst)
        abcfill0 = abcfill24(fill, 0)
        abcfill1 = abcfill24(fill, crdc)
    end        
    return("|:" * fp[vol] * abctime * "[1" * abcfill0 * ":|2" * abcfill1 * "\n")
end


abctune_multitime_displace (generic function with 1 method)

In [38]:
#put it all together, output abc notation
import Dates
function abcsolo_multitime_displace(
        timesig::Int,
  len::Int,
  basstrans::AugMatrix,
  filltrans::AugDblMatrix,
  dyntrans::AugDblMatrix,
  pattrans::AugMatrix,
  insttrans::AugMatrix,
  feel::String;
        variation="rock"
    )
    s = ""
    curdyn = markovrand(dyn.init)
    curpat = markovrand(pattrans.init)
    curinst = markovrand(insttrans.init)
    for i = 1:len
        vol = div(curdyn, 2) + 1
        if (timesig == 8) || (timesig == 12)
            bass = ghostbassgen_pairs(24, basstrans)
        elseif (timesig == 16) || (timesig == 24)
            bass = ghostbassgen(24, basstrans)
        end
        pat = curpat + 1
        inst = curinst + 1
        if (timesig == 8) || (timesig == 16)
            fill = fillgen(16, filltrans)
        elseif (timesig == 12) 
            fill = fillgen(12, filltrans)
        elseif (timesig == 24)
            fill = fillgen(24, filltrans)
        end
        crdc = 2 * mod(curdyn, 2) - 1
        s = s * abctune_multitime_displace(timesig, vol, bass, pat, inst, fill, crdc, variation=variation)
        if i + 1 < len
            curdyn = markovrand(dyntrans.mat1[:, curdyn + 1])
        else
            curdyn = markovrand(dyntrans.mat2[:, curdyn + 1])
        end
        curpat = markovrand(pattrans.mat[:, curpat + 1])
        curinst = markovrand(insttrans.mat[:, curinst + 1])
    end
    s = insert(s, findlast(occursin("acbdefgABCDEFG"), s)-1, "!fermata!") #final fermata
    s = replace(s, "[]" => "z") #rest
    s = makeppheader("Computer-generated \\`Etude for Solo Drum Kit", String(@functionName) * "/" * variation, "$(Dates.now())", "4/4", feel, 80) * s * "|]\n"
    return(s)
end


abcsolo_multitime_displace (generic function with 1 method)

In [39]:
dynMatrix = [
    0 0 0.2e0 0.2e0 0.1e0 0.1e0 0 0 0 0 0 0; 
    0 0 0.2e0 0.1e0 0.1e0 0.1e0 0 0 0 0 0 0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.2e0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0; 
    0 0 0.1e0 0.2e0 0 0 0.2e0 0.1e0 0 0 0.1e0 0.2e0; 
    0 0 0.2e0 0.2e0 0 0 0.2e0 0.1e0 0 0 0.2e0 0.1e0; 
    0.1e0 0.2e0 0 0 0.1e0 0.2e0 0 0 0.2e0 0.2e0 0 0; 
    0.2e0 0.1e0 0 0 0.1e0 0.2e0 0 0 0.2e0 0.1e0 0 0; 
    0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0;
    0.2e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0; 
    0 0 0 0 0 0 0.1e0 0.1e0 0.1e0 0.2e0 0 0; 
    0 0 0 0 0 0 0.1e0 0.1e0 0.2e0 0.2e0 0 0]


dynFinal = [
    0.1e0 0.2e0 0.2e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0 0; 
    0.2e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0 0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.2e0 0.1e0 0.2e0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0; 
    0.2e0 0.1e0 0.2e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0; 
    0 0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.2e0; 
    0 0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.2e0 0.2e0 0.1e0]


dynInitial = [0.5e-1,0.5e-1,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.5e-1,0.5e-1]
#pp>, pp<, p>, p<, mp>, mp<, mf>, mf<, f>, f<, ff>, ff<

dyn = AugDblMatrix(dynMatrix, dynFinal, dynInitial)

12×12 Matrix{Float64}:
 0.0  0.0  0.2  0.2  0.1  0.1  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.2  0.1  0.1  0.1  0.0  0.0  0.0  0.0  0.0  0.0
 0.2  0.2  0.0  0.0  0.2  0.1  0.1  0.1  0.1  0.2  0.1  0.2
 0.2  0.2  0.0  0.0  0.2  0.1  0.1  0.1  0.2  0.1  0.2  0.1
 0.0  0.0  0.1  0.2  0.0  0.0  0.2  0.1  0.0  0.0  0.1  0.2
 0.0  0.0  0.2  0.2  0.0  0.0  0.2  0.1  0.0  0.0  0.2  0.1
 0.1  0.2  0.0  0.0  0.1  0.2  0.0  0.0  0.2  0.2  0.0  0.0
 0.2  0.1  0.0  0.0  0.1  0.2  0.0  0.0  0.2  0.1  0.0  0.0
 0.1  0.2  0.1  0.2  0.1  0.1  0.1  0.2  0.0  0.0  0.2  0.2
 0.2  0.1  0.2  0.1  0.1  0.1  0.1  0.2  0.0  0.0  0.2  0.2
 0.0  0.0  0.0  0.0  0.0  0.0  0.1  0.1  0.1  0.2  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.1  0.1  0.2  0.2  0.0  0.0
12×12 Matrix{Float64}:
 0.1  0.2  0.2  0.2  0.1  0.1  0.1  0.1  0.1  0.2  0.0  0.0
 0.2  0.1  0.2  0.1  0.1  0.1  0.1  0.1  0.2  0.1  0.0  0.0
 0.2  0.2  0.0  0.0  0.2  0.1  0.2  0.1  0.2  0.2  0.1  0.2
 0.2  0.2  0.0  0.0  0.2  0.1  0.2  0.1  0.2  0.1  0.2

In [40]:
#0 = rest, 1 = bass, 2 = LH
                #00    #01  #02   #10  #11  #12     #zeros
rockBassMatrix = [0.1 1//6  0   1//6  0.5    0      0 0 0 #00
                    0   1//3  0     0    0    0     0 0 0  #01
                    0   1//6  0.1   1//6  0.5   0.1 0 0 0  #02
                    0.45 1//3  0.45  1//3 0    0.45 0 0 0  #10
                    0.45 0    0.45  1//3  0    0.45 0 0 0  #11
                    0  0    0      0   0    0       0 0 0    #12
                    zeros(3,9)]                 #zeros

rockBassInitial = [0.1,0, 0.1, 0.4, 0.4, 0, 0, 0, 0]

rockBass = AugMatrix(rockBassMatrix, rockBassInitial)

9×9 Matrix{Float64}:
 0.1   0.166667  0.0   0.166667  0.5  0.0   0.0  0.0  0.0
 0.0   0.333333  0.0   0.0       0.0  0.0   0.0  0.0  0.0
 0.0   0.166667  0.1   0.166667  0.5  0.1   0.0  0.0  0.0
 0.45  0.333333  0.45  0.333333  0.0  0.45  0.0  0.0  0.0
 0.45  0.0       0.45  0.333333  0.0  0.45  0.0  0.0  0.0
 0.0   0.0       0.0   0.0       0.0  0.0   0.0  0.0  0.0
 0.0   0.0       0.0   0.0       0.0  0.0   0.0  0.0  0.0
 0.0   0.0       0.0   0.0       0.0  0.0   0.0  0.0  0.0
 0.0   0.0       0.0   0.0       0.0  0.0   0.0  0.0  0.0
9-element Vector{Float64}:
 0.1
 0.0
 0.1
 0.4
 0.4
 0.0
 0.0
 0.0
 0.0

In [41]:
rockNoGhostBMatrix = [0.1e0 1//3 1//3 1; 0 1//3 0 0; 0.45e0 1//3 1//3 0; 0.45e0 0 1//3 0]

rockNoGhostBInitial = [0.1e0,0, 0,0.5e0,0.4e0, 0, 0, 0, 0]

rockNoGhostB = AugMatrix([rockNoGhostBMatrix[1:2, 1:2] zeros(2,1) rockNoGhostBMatrix[1:2, 3:4] zeros(2, 4)
    zeros(1, 9)
    rockNoGhostBMatrix[3:4, 1:2] zeros(2,1) rockNoGhostBMatrix[3:4, 3:4] zeros(2, 4)
    zeros(4, 9)
    ],
    rockNoGhostBInitial)

9×9 Matrix{Float64}:
 0.1   0.333333  0.0  0.333333  1.0  0.0  0.0  0.0  0.0
 0.0   0.333333  0.0  0.0       0.0  0.0  0.0  0.0  0.0
 0.0   0.0       0.0  0.0       0.0  0.0  0.0  0.0  0.0
 0.45  0.333333  0.0  0.333333  0.0  0.0  0.0  0.0  0.0
 0.45  0.0       0.0  0.333333  0.0  0.0  0.0  0.0  0.0
 0.0   0.0       0.0  0.0       0.0  0.0  0.0  0.0  0.0
 0.0   0.0       0.0  0.0       0.0  0.0  0.0  0.0  0.0
 0.0   0.0       0.0  0.0       0.0  0.0  0.0  0.0  0.0
 0.0   0.0       0.0  0.0       0.0  0.0  0.0  0.0  0.0
9-element Vector{Float64}:
 0.1
 0.0
 0.0
 0.5
 0.4
 0.0
 0.0
 0.0
 0.0

In [42]:
rockFillLeftToRight = [
    0.2e0 0.1e0 0.2e0 0 0.3e0 0.2e0; 
    0.4e0 0.2e0 0 0.3e0 0.3e0 0.2e0; 
    0.1e0 0.4e0 0.2e0 0.1e0 0.1e0 0.2e0; 
    0.1e0 0.1e0 0.4e0 0.3e0 0.1e0 0.2e0; 
    0.1e0 0.1e0 0.2e0 0.2e0 0.1e0 0.2e0; 
    0.1e0 0.1e0 0 0.1e0 0.1e0 0]

rockFillRightToLeft = [
    0.4e0 0.2e0 0.1e0 0.1e0 0.3e0 0
    0.2e0 0.4e0 0.2e0 0.2e0 0.2e0 0
    0.1e0 0 0.4e0 0.1e0 0.1e0 0
    0 0.1e0 0.1e0 0.4e0 0.1e0 0
    0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0
    0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.10e1]


rockFillInitial = [0.4e0,0.4e0,0.1e0,0.1e0,0,0] #snare, high, mid, low, kick, rest

rockFill= AugDblMatrix(rockFillLeftToRight, rockFillRightToLeft, rockFillInitial)

6×6 Matrix{Float64}:
 0.2  0.1  0.2  0.0  0.3  0.2
 0.4  0.2  0.0  0.3  0.3  0.2
 0.1  0.4  0.2  0.1  0.1  0.2
 0.1  0.1  0.4  0.3  0.1  0.2
 0.1  0.1  0.2  0.2  0.1  0.2
 0.1  0.1  0.0  0.1  0.1  0.0
6×6 Matrix{Float64}:
 0.4  0.2  0.1  0.1  0.3  0.0
 0.2  0.4  0.2  0.2  0.2  0.0
 0.1  0.0  0.4  0.1  0.1  0.0
 0.0  0.1  0.1  0.4  0.1  0.0
 0.2  0.1  0.1  0.1  0.2  0.0
 0.1  0.2  0.1  0.1  0.1  1.0
6-element Vector{Float64}:
 0.4
 0.4
 0.1
 0.1
 0.0
 0.0

In [43]:
rockPatMatrix = [
    #8  Q   b   +
    0   0.4 0.4 0.4 #8 
    0.4 0   0.2 0.2   #Q
    0.2 0.2 0   0.4 #b
    0.4 0.4 0.4 0 #+
    ]



rockPatInitial = [0.4; 0.4; 0.2; 0]

rockPat = AugMatrix(rockPatMatrix, rockPatInitial)

4×4 Matrix{Float64}:
 0.0  0.4  0.4  0.4
 0.4  0.0  0.2  0.2
 0.2  0.2  0.0  0.4
 0.4  0.4  0.4  0.0
4-element Vector{Float64}:
 0.4
 0.4
 0.2
 0.0

In [44]:
rockInstMatrix = [
   #HH  RC  RB  ST 
    0.1 0.4 0.4   0.4   #HH
    0.6 0.1 0.4   0.4   #RC
    0.2 0.4 0.1   0.2   #RB
    0.1 0.1 0.1   0     #ST
]

rockInstInitial = [0.5; 0.3; 0; 0.2]

rockInst = AugMatrix(rockInstMatrix, rockInstInitial)

4×4 Matrix{Float64}:
 0.1  0.4  0.4  0.4
 0.6  0.1  0.4  0.4
 0.2  0.4  0.1  0.2
 0.1  0.1  0.1  0.0
4-element Vector{Float64}:
 0.5
 0.3
 0.0
 0.2

In [58]:
rocksolo = abcsolo_multitime_displace(8, 4, rockBass, rockFill, dyn, rockPat, rockInst, "Rock 8th feel")
print(rocksolo)

%%abc-include percussions-JBH.abh

I:linebreak $

#ifdef MIDI
#define !>! "[I:volinc 50]"
#define !Q! "[I:vol 20]" 
                %%%ghost
#define !U! "[I:vol 40]" 
                %%%unaccent
#define !N! "[I:volinc -25]" 
            %%%tenuto for trash hat
#else
#define !N! "\"^_\"" 
                %%%tenuto
#define !Q! !(.!!).! 
                %%%ghost
#define !U! !anti!  
                %%%unaccent
#endif

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:#abcsolo_multitime_displace#26/rock
O:2022-01-28T20:14:36.879
M:4/4
L:1/8
Q:"Rock 8th feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
|:!mp![^e]y[^eF]y[^e][^eF]y [^eF]y[^e]y[^ecF]/2[F/2][^e]y |
[^eF]/2[F/2][^e]y[^ecF]y[^eF]/2[F/2] [^e]/2[!Q!c/2][^eF]y[^ecF]y[^eF]/2[F/2] |
[^e]/2[!Q!c/2]

In [59]:
#write a string to a file
open(f->print(f, rocksolo), "rockghosttest.abp", "w")

#Converts ABP file to score and MIDI and displays/plays it
abptest("rockghosttest.abp")

abcm2ps-8.14.12 (2021-07-14)
File rockghosttest-ps.abc
File percussions-JBH.abh
Output written on rockghosttest.xhtml (1 page, 1 title, 47954 bytes)


rockghosttest-ps.abc:29:47: warning: Line overfull (631pt of 628pt)


4.59 June 27 2021 abc2midi
writing MIDI file rockghosttest.mid
Warning in line-char 41-0 : Track 1 Bar 19 has 9/2 units instead of 4 in repeat


Process(`cmd /C start rockghosttest.xhtml`, ProcessExited(0))

In [36]:
#0 = rest, 1 = bass, 2 = LH
                #00    #01  #02   #10  #11   #12  #zeros
funkBassMatrix = [0.1e0 0.2  0     0.1  1//3 0   0 0 0 #00
                    0.2 0.2  0.2   0.2  1//3 0.1    0 0 0  #01
                    0.2 0.2  0.2   0.2  1//3 0.1    0 0 0  #02
                    0.1 0.2  0.2   0.1  0    0.3 0 0 0  #10
                    0.2 0    0.2   0.2  0    0.2 0 0 0  #11
                    0.2 0.2  0.2   0.2  0    0.3 0 0 0    #12
                    zeros(3,9)]                 #zeros

funkBassInitial = [0.1, 0.2, 0.1, 0.2, 0.2, 0.2, 0, 0, 0]

funkBass = AugMatrix(funkBassMatrix, funkBassInitial)

9×9 Matrix{Float64}:
 0.1  0.2  0.0  0.1  0.333333  0.0  0.0  0.0  0.0
 0.2  0.2  0.2  0.2  0.333333  0.1  0.0  0.0  0.0
 0.2  0.2  0.2  0.2  0.333333  0.1  0.0  0.0  0.0
 0.1  0.2  0.2  0.1  0.0       0.3  0.0  0.0  0.0
 0.2  0.0  0.2  0.2  0.0       0.2  0.0  0.0  0.0
 0.2  0.2  0.2  0.2  0.0       0.3  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0
9-element Vector{Float64}:
 0.1
 0.2
 0.1
 0.2
 0.2
 0.2
 0.0
 0.0
 0.0

In [ ]:
funkNoGhostBMatrix = [0.1e0 1//3 0.25e0 0.5e0; 0.3e0 1//3 0.25e0 0.5e0; 0.3e0 1//3 0.25e0 0; 0.3e0 0 0.25e0 0]

funkNoGhostBInitial = [0.1e0, 0.1e0, 0, 0.4e0, 0.4e0, 0, 0, 0, 0 ]

funkNoGhostB = AugMatrix([funkNoGhostBMatrix[1:2, 1:2] zeros(2,1) funkNoGhostBMatrix[1:2, 3:4] zeros(2, 4)
    zeros(1, 9)
    funkNoGhostBMatrix[3:4, 1:2] zeros(2,1) funkNoGhostBMatrix[3:4, 3:4] zeros(2, 4)
    zeros(4, 9)
    ],
    funkNoGhostBInitial)

In [ ]:
funkFillLeftToRight = [0.2e0 0.1e0 0.1e0 0 0.3e0 0.2e0; 0.4e0 0.2e0 0 0.3e0 0.3e0 0.2e0; 0.1e0 0.4e0 0.2e0 0.1e0 0.1e0 0.2e0; 0.1e0 0.1e0 0.4e0 0.3e0 0.1e0 0.2e0; 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.1e0; 0.1e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0]

funkFillRightToLeft = [0.4e0 0.2e0 0.1e0 0.1e0 0.3e0 0.2e0; 0.2e0 0.4e0 0.2e0 0.2e0 0.2e0 0.2e0; 0.1e0 0 0.4e0 0.1e0 0.1e0 0.2e0; 0 0.1e0 0.1e0 0.4e0 0.1e0 0.2e0; 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0; 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0]


funkFillInitial = [0.3e0,0.3e0,0.1e0,0.1e0,0.1e0,0.1e0]

funkFill = AugDblMatrix(funkFillLeftToRight, funkFillRightToLeft, funkFillInitial)

In [ ]:
funkPatMatrix = [
    #8  Q   b   +
    0   0.2 0.2 0.2 #8 
    0.2 0   0.4 0.4   #Q
    0.2 0.2 0   0.4 #b
    0.6 0.6 0.4 0 #+
    ]

funkPatInitial = [0; 0.2; 0.4; 0.4]

funkPat = AugMatrix(funkPatMatrix, funkPatInitial)

In [ ]:
funkInstMatrix = [
   #HH  RC  RB  ST 
    0.1 0.4 0.4   0.4   #HH
    0.6 0.1 0.4   0.4   #RC
    0.2 0.4 0.1   0.2   #RB
    0.1 0.1 0.1   0     #ST
]

funkInstInitial = [0.4; 0.4; 0.2; 0]

funkInst = AugMatrix(funkInstMatrix, funkInstInitial)

In [ ]:
funksolo = abcsolo8patinst16ghost(4, funkBass, funkFill, dyn, funkPat, funkInst, "Funk 16th feel")
print(funksolo)

In [ ]:
writetune("funkghosttest.abp", funksolo)

abptest("funkghosttest.abp")

In [ ]:
halffunksolo = abcsolo8patinst16ghost(4, 0.5*funkBass+0.5*rockBass, 0.5*funkFill+0.5*rockFill, dyn, 0.5*funkPat+0.5*rockPat, 0.5*funkInst+0.5*rockInst, "Rock/Funk 16th feel")
print(halffunksolo)

In [ ]:
writetune("halffunktest.abp", halffunksolo)

abptest("halffunktest.abp")

In [ ]:
#0 = rest, 1 = bass
discoNoGhostBMatrix = 
    #00   #01  #10      #11
    [0    0    0.5   0.5; 
    0     0    0.5   0.5; 
    0.5   1    0      0; 
    0.5   0    0      0]

discoNoGhostBInitial = [0, 0, 0, 0.5, 0.5, 0, 0, 0, 0 ]

discoNoGhostB = AugMatrix([discoNoGhostBMatrix[1:2, 1:2] zeros(2,1) discoNoGhostBMatrix[1:2, 3:4] zeros(2, 4)
    zeros(1, 9)
    discoNoGhostBMatrix[3:4, 1:2] zeros(2,1) discoNoGhostBMatrix[3:4, 3:4] zeros(2, 4)
    zeros(4, 9)
    ],
    discoNoGhostBInitial)

In [ ]:
discoPatMatrix = [
    #Q  8   b   +
    0   0.2 0.2 0.2 #Q 
    0.2 0   0.4 0.4   #8
    0.2 0.2 0   0.4 #b
    0.6 0.6 0.4 0 #+
    ]

discoPatInitial = [0; 0.5; 0.5; 0]

discoPat = AugMatrix(discoPatMatrix, discoPatInitial)

In [ ]:
discoInstMatrix = [
   #HH  RC  RB  ST 
    0.1 0.5 0.6   0.4   #HH
    0.3 0.1 0.3   0.2   #RC
    0.6 0.4 0.1   0.4   #RB
    0    0  0   0     #ST
]

discoInstInitial = [0.6; 0.2; 0.2; 0]

discoInst = AugMatrix(discoInstMatrix, discoInstInitial)

In [ ]:
discoFillLeftToRight = [0.2e0 0.1e0 0.1e0 0 0.3e0 0.2e0; 0.4e0 0.2e0 0 0.3e0 0.3e0 0.2e0; 0.1e0 0.4e0 0.2e0 0.1e0 0.1e0 0.2e0; 0.1e0 0.1e0 0.4e0 0.3e0 0.1e0 0.2e0; 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.1e0; 0.1e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0]

discoFillRightToLeft = [0.4e0 0.2e0 0.1e0 0.1e0 0.3e0 0.2e0; 0.2e0 0.4e0 0.2e0 0.2e0 0.2e0 0.2e0; 0.1e0 0 0.4e0 0.1e0 0.1e0 0.2e0; 0 0.1e0 0.1e0 0.4e0 0.1e0 0.2e0; 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0; 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0]


discoFillInitial = [0.3e0,0.3e0,0.1e0,0.1e0,0.1e0,0.1e0]

discoFill = AugDblMatrix(discoFillLeftToRight, discoFillRightToLeft, discoFillInitial)

In [ ]:
dancedynMatrix = [
    0 0 0.2e0 0.2e0 0.1e0 0.1e0 0 0 0 0 0 0; 
    0 0 0.2e0 0.1e0 0.1e0 0.1e0 0 0 0 0 0 0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.2e0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0; 
    0 0 0.1e0 0.2e0 0 0 0.2e0 0.1e0 0 0 0.1e0 0.2e0; 
    0 0 0.2e0 0.2e0 0 0 0.2e0 0.1e0 0 0 0.2e0 0.1e0; 
    0.1e0 0.2e0 0 0 0.1e0 0.2e0 0 0 0.2e0 0.2e0 0 0; 
    0.2e0 0.1e0 0 0 0.1e0 0.2e0 0 0 0.2e0 0.1e0 0 0; 
    0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0;
    0.2e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0; 
    0 0 0 0 0 0 0.1e0 0.1e0 0.1e0 0.2e0 0 0; 
    0 0 0 0 0 0 0.1e0 0.1e0 0.2e0 0.2e0 0 0]


dancedynFinal = [
    #pp>, pp<, p>, p<, mp>, mp<, mf>, mf<, f>, f<, ff>, ff<
    zeros(1, 12); 
    zeros(1, 12); 
    0   0   0   0      0   0   0   0      0   0   0   0; 
    0   0   0   0      0.4 0.2 0.4 0.2    0.4 0.3 0.3 0.3; 
    zeros(1, 12); 
    zeros(1, 12);    
    zeros(1, 12); 
    zeros(1, 12); 
    0.2 0.3 0.2 0.3    0.2 0.2 0.2 0.2    0   0   0.2 0.2; 
    0.3 0.2 0.3 0.2    0.2 0.2 0.2 0.2    0   0   0.2 0.2; 
    0.2 0.3 0.2 0.3    0.1 0.2 0.1 0.2    0.2 0.4 0.1 0.2; 
    0.3 0.2 0.3 0.2    0.1 0.2 0.1 0.2    0.4 0.3 0.2 0.1]


dancedynInitial = [0,0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1,0.1]
#pp>, pp<, p>, p<, mp>, mp<, mf>, mf<, f>, f<, ff>, ff<

dancedyn = AugDblMatrix(dancedynMatrix, dancedynFinal, dancedynInitial)

In [ ]:
discosolo = abcsolo8patinst16ghost(4, discoNoGhostB, funkFill, dancedyn, discoPat, discoInst, "Disco 16th feel", variation="disco")
print(discosolo)

In [ ]:
writetune("disconoghosttest.abp", discosolo)

abptest("disconoghosttest.abp")

In [ ]:
#0 = rest, 1 = bass, 2 = LH
                #00    #01  #02   #10  #11  #12  #zeros
discoBassMatrix = [0   0    0     0.2  1//3 0   0 0 0 #00
                    0  0    0     0.4  1//3 0.5    0 0 0  #01
                    0  0    0     0.4  1//3 0.5    0 0 0  #02
                    0.2 0.5  1//3   0    0    0   0 0 0  #10
                    0.4 0    1//3   0    0    0   0 0 0  #11
                    0.4 0.5  1//3   0    0    0   0 0 0    #12
                    zeros(3,9)]                 #zeros

sum(discoBassMatrix, dims=1)

discoBassInitial = [0, 0, 0, 1//3, 1//3, 1//3, 0, 0, 0]

discoBass = AugMatrix(discoBassMatrix, discoBassInitial)

In [ ]:
discosolo = abcsolo8patinst16ghost(4, discoBass, funkFill, dancedyn, discoPat, discoInst, "Disco 16th feel", variation="disco")
print(discosolo)

In [ ]:
writetune("discoghosttest.abp", discosolo)

abptest("discoghosttest.abp")

In [38]:
#0 = rest, 1 = bass, 2 = LH
                #00    #01  #02   #10  #11  #12  #zeros
shuffleBassMatrix = [0.1e0 0.2  0.2   0.25 1    0.25 0 0 0 #00
                    0   0.2  0.1   0    0    0    0 0 0  #01
                    0   0.2  0.1   0    0    0    0 0 0  #02
                    0.3 0.2  0.2   0.25 0    0.25 0 0 0  #10
                    0.3 0    0.2   0.25 0    0.25 0 0 0  #11
                    0.3 0.2  0.2   0.25 0    0.25 0 0 0    #12
                    zeros(3,9)]                 #zeros

shuffleBassInitial = [0.1,0,0, 0.3,0.3, 0.3, 0, 0, 0]

shuffleBass = AugMatrix(shuffleBassMatrix, shuffleBassInitial)

9×9 Matrix{Float64}:
 0.1  0.2  0.2  0.25  1.0  0.25  0.0  0.0  0.0
 0.0  0.2  0.1  0.0   0.0  0.0   0.0  0.0  0.0
 0.0  0.2  0.1  0.0   0.0  0.0   0.0  0.0  0.0
 0.3  0.2  0.2  0.25  0.0  0.25  0.0  0.0  0.0
 0.3  0.0  0.2  0.25  0.0  0.25  0.0  0.0  0.0
 0.3  0.2  0.2  0.25  0.0  0.25  0.0  0.0  0.0
 0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0
 0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0
 0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0
9-element Vector{Float64}:
 0.1
 0.0
 0.0
 0.3
 0.3
 0.3
 0.0
 0.0
 0.0

In [64]:
halfshufflesolo = abcsolo_multitime_ghost(24, 4, shuffleBass, rockFill, dyn, rockPat, rockInst, "Half-time shuffle feel")
print(halfshufflesolo)

%%abc-include percussions-JBH.abh

I:linebreak $

#ifdef MIDI
#define !>! "[I:volinc 50]"
#define !Q! "[I:vol 20]" 
                %%%ghost
#define !U! "[I:vol 40]" 
                %%%unaccent
#define !N! "[I:volinc -25]" 
            %%%tenuto for trash hat
#else
#define !N! "\"^_\"" 
                %%%tenuto
#define !Q! !(.!!).! 
                %%%ghost
#define !U! !anti!  
                %%%unaccent
#endif

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:#abcsolo_multitime_ghost#59/rock
O:2022-01-01T19:56:25.584
M:4/4
L:1/8
Q:"Half-time shuffle feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
|:!f!(6:4!N![_eF]/2z/2[!Q!c]/2!N![_e]/2z/2z/2 (6:4!N![_ecF]/2z/2[F]/2!N![_e]/2z/2z/2 (6:4!N![_e]/2z/2z/2!N![_eF]/2z/2z/2 (6:4!N![_ec]/2z/2z/2!N![_e

In [65]:
#write a string to a file
open(f->print(f, halfshufflesolo), "halfshuffleghosttest.abp", "w")

#Converts ABP file to score and MIDI and displays/plays it
abptest("halfshuffleghosttest.abp")

abcm2ps-8.14.12 (2021-07-14)
File halfshuffleghosttest-ps.abc
File percussions-JBH.abh
Output written on halfshuffleghosttest.xhtml (2 pages, 1 title, 81566 bytes)
4.59 June 27 2021 abc2midi
Warning in line-char 25-30 : Different length notes in tuple
Warning in line-char 25-68 : Different length notes in tuple
Warning in line-char 25-71 : Different length notes in tuple
Warning in line-char 25-101 : Different length notes in tuple
Warning in line-char 25-129 : Different length notes in tuple
Warning in line-char 25-132 : Different length notes in tuple
Warning in line-char 25-160 : Different length notes in tuple
Warning in line-char 25-163 : Different length notes in tuple
Warning in line-char 25-187 : Different length notes in tuple
Warning in line-char 25-190 : Different length notes in tuple
Warning in line-char 25-219 : Different length notes in tuple
Warning in line-char 25-222 : Different length notes in tuple
Warning in line-char 25-246 : Different length notes in tuple
Warnin

halfshuffleghosttest-ps.abc:28:8: warning: Line overfull (637pt of 628pt)


Process(`cmd /C start halfshuffleghosttest.xhtml`, ProcessExited(0))

In [75]:
#0 = rest, 1 = bass, 2 = LH
                #00    #01  #02   #10  #11  #12     #zeros
rock8BassMatrix = [0.1 1//6  0   1//3  0.5    0      0 0 0 #00
                    0   1//3  0     0    0    0     0 0 0  #01
                    0   1//6  0.1   0  0.5   0.1 0 0 0  #02
                    0.9 1//3  0.45  2//3 0    0.45 0 0 0  #10
                    0    0    0.45  0     0    0.45 0 0 0  #11
                    0  0    0      0   0    0       0 0 0    #12
                    zeros(3,9)]                 #zeros

rock8BassInitial = [0.2, 0, 0, 0.8, 0, 0, 0, 0, 0]

rock8Bass = AugMatrix(rock8BassMatrix, rock8BassInitial)

9×9 Matrix{Float64}:
 0.1  0.166667  0.0   0.333333  0.5  0.0   0.0  0.0  0.0
 0.0  0.333333  0.0   0.0       0.0  0.0   0.0  0.0  0.0
 0.0  0.166667  0.1   0.0       0.5  0.1   0.0  0.0  0.0
 0.9  0.333333  0.45  0.666667  0.0  0.45  0.0  0.0  0.0
 0.0  0.0       0.45  0.0       0.0  0.45  0.0  0.0  0.0
 0.0  0.0       0.0   0.0       0.0  0.0   0.0  0.0  0.0
 0.0  0.0       0.0   0.0       0.0  0.0   0.0  0.0  0.0
 0.0  0.0       0.0   0.0       0.0  0.0   0.0  0.0  0.0
 0.0  0.0       0.0   0.0       0.0  0.0   0.0  0.0  0.0
9-element Vector{Float64}:
 0.2
 0.0
 0.0
 0.8
 0.0
 0.0
 0.0
 0.0
 0.0

In [100]:
shufflesolo = abcsolo_multitime_ghost(12, 4, rock8Bass, rockFill, dyn, rockPat, rockInst, "Shuffle feel")
print(shufflesolo)

%%abc-include percussions-JBH.abh

I:linebreak $

#ifdef MIDI
#define !>! "[I:volinc 50]"
#define !Q! "[I:vol 20]" 
                %%%ghost
#define !U! "[I:vol 40]" 
                %%%unaccent
#define !N! "[I:volinc -25]" 
            %%%tenuto for trash hat
#else
#define !N! "\"^_\"" 
                %%%tenuto
#define !Q! !(.!!).! 
                %%%ghost
#define !U! !anti!  
                %%%unaccent
#endif

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:#abcsolo_multitime_ghost#101/rock
O:2022-01-01T20:20:51.644
M:4/4
L:1/8
Q:"Shuffle feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
|:!mf!(3:2!>![^eF]yz!U![^eF]y (3:2!>![^ecF]yz!U![^eF]y (3:2!>![^e]yz!U![^e]y (3:2!>![^ecF]yz!U![^eF]y |
(3:2!>![^eF]yz!U![^eF]y (3:2!>![^ecF]yz!U![^e]y (3:2

In [101]:
#write a string to a file
open(f->print(f, shufflesolo), "shuffleghosttest.abp", "w")

#Converts ABP file to score and MIDI and displays/plays it
abptest("shuffleghosttest.abp")

abcm2ps-8.14.12 (2021-07-14)
File shuffleghosttest-ps.abc
File percussions-JBH.abh
Output written on shuffleghosttest.xhtml (1 page, 1 title, 51096 bytes)
4.59 June 27 2021 abc2midi
writing MIDI file shuffleghosttest.mid
Warning in line-char 41-0 : Track 1 Bar 19 has 9/2 units instead of 4 in repeat


shuffleghosttest-ps.abc:28:109: warning: Line overfull (628pt of 628pt)


Process(`cmd /C start shuffleghosttest.xhtml`, ProcessExited(0))